# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.



---
### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!). 

![Sample Dog Output](images/sample_dog_output.png)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

Make sure that you've downloaded the required human and dog datasets:

**Note: if you are using the Udacity workspace, you *DO NOT* need to re-download these - they can be found in the `/data` folder as noted in the cell below.**

* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dog_images`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home directory, at location `/lfw`.  

*Note: If you are using a Windows machine, you are encouraged to use [7zip](http://www.7-zip.org/) to extract the folder.*

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [ ]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         name = os.path.join(dirname, filename)
#         print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
from glob import glob ## for paths
d = "/kaggle/input/dog-breed-and-human-dataset"

# load filenames for human and dog images
human_files = np.array(glob(d+"/lfw/*/*"))
dog_files = np.array(glob(d+"/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

<a id='step1'></a>
## Step 1: Detect Humans

In this section, we use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  

OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.  In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
# import cv2                
# import matplotlib.pyplot as plt                        
# %matplotlib inline                               

# # extract pre-trained face detector
# face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# # load color (BGR) image
# img = cv2.imread(human_files[2])
# # convert BGR image to grayscale
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # find faces in image
# faces = face_cascade.detectMultiScale(gray)

# # print number of faces detected in the image
# print('Number of faces detected:', len(faces))

In [ ]:
s = cv2.imread(dog_files[0])
print(s.shape)

In [ ]:
# # get bounding box for each detected face
# for (x,y,w,h) in faces:
#     # add bounding box to color image
#     cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    
# # convert BGR image to RGB for plotting
# cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# # display the image, along with bounding box
# plt.imshow(cv_rgb)
# plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# # returns "True" if face is detected in image stored at img_path
# def face_detector(img_path):
#     img = cv2.imread(img_path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray)
#     return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 
(You can print out your results and/or write your percentages in this cell)

In [ ]:
# from tqdm import tqdm

# human_files_short = human_files[:100]
# dog_files_short = dog_files[:100]
# # print(human_files_short)

# #-#-# Do NOT modify the code above this line. #-#-#

# ### TODO: Test the performance of the dog_detector function
# ### on the images in human_files_short and dog_files_short.

# result_human_fd = [face_detector(x) for x in human_files_short]
# result_dog_fd = [face_detector(x) for x in dog_files_short]

# tot_human_fd = 0
# tot_dog_fd = 0

# for i in result_human_fd:
#     tot_human_fd+= 1 if i else 0
# for i in result_dog_fd:
#     tot_dog_fd+= 0 if i else 1

# human_percent_fd = (tot_human_fd/100 ) * len(human_files_short)
# dog_percent_fd = (tot_dog_fd/100) * len(dog_files_short)


# print("{} % human images are correctly classified".format(human_percent_fd))
# print("{} % dogs images are correctly classified".format(dog_percent_fd))

# ## TODO: Test the performance of the face_detector algorithm 
# ## on the images in human_files_short and dog_files_short.
# # 


We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).  Please use the code cell below to design and test your own face detection algorithm.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

In [ ]:
### (Optional) 
### TODO: Test performance of anotherface detection algorithm.
### Feel free to use as many code cells as needed.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [ ]:
# import torch
# import torchvision.models as models

# # define VGG16 model
# VGG16 = models.vgg16(pretrained=True)

# # check if CUDA is available
# use_cuda = torch.cuda.is_available()

# # move model to GPU if CUDA is available
# if use_cuda:
#     VGG16 = VGG16.cuda()

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  The output should always be an integer between 0 and 999, inclusive.

Before writing the function, make sure that you take the time to learn  how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
# from PIL import Image
# import torchvision.transforms as transforms
# import torchvision.models as models


# vgg16 = models.vgg16(pretrained = True)
# print(vgg16)


# def VGG16_predict(img_path):
#     '''
#     Use pre-trained VGG-16 model to obtain index corresponding to 
#     predicted ImageNet class for image at specified path
    
#     Args:
#         img_path: path to an image
        
#     Returns:
#         Index corresponding to VGG-16 model's prediction
#     '''
    
#     ## TODO: Complete the function.
#     ## Load and pre-process an image from the given img_path
#     ## Return the *index* of the predicted class for that image
    
#     data_transform = transforms.Compose([transforms.Scale((224,224)), 
#                                       transforms.ToTensor()])
#     #open_img:
    
# #     x = cv2.imread(img_path)
#     #bgr to rgb

#     ## resize image
#     x=Image.open(img_path)
#     x= data_transform(x)
    

# #     if torch.cuda.is_available():
# #         x = x.cuda()
# #     x = torch.Tensor(x)
#     x.unsqueeze_(0)
    
#     global vgg16
#     vgg16.eval()
#     outputs = vgg16(x)
#     _,output = torch.max(outputs.data,1)
# #     print(output)
    
    
#     return output # predicted class index

### (IMPLEMENTATION) Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

Use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
# ### returns "True" if a dog is detected in the image stored at img_path
# def dog_detector(img_path):
#     ## TODO: Complete the function.
#     output = VGG16_predict(img_path)
#     status = True if (output>150 and output<269) else False
#     return status

### (IMPLEMENTATION) Assess the Dog Detector

__Question 2:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ 


In [ ]:
# result_dog_dd = dog_detector(dog_files_short[55])
# print(result_dog_dd)

In [ ]:
# ## TODO: Test the performance of the dog_detector function
# ## on the images in human_files_short and dog_files_short.

# result_human_dd = [dog_detector(x) for x in human_files_short]
# result_dog_dd = [dog_detector(x) for x in dog_files_short]

# tot_human_dd = 0
# tot_dog_dd = 0

# for i in result_human_dd:
#     tot_human_dd+= 0 if i else 1
# for i in result_dog_dd:
#     tot_dog_dd+= 1 if i else 0

# human_percent_dd = (tot_human_dd/100 ) * len(human_files_short)
# dog_percent_dd = (tot_dog_dd/100) * len(dog_files_short)

# print("{} % human images are correctly classified".format(human_percent_dd))
# print("{} % dogs images are correctly classified".format(dog_percent_dd))

We suggest VGG-16 as a potential network to detect dog images in your algorithm, but you are free to explore other pre-trained networks (such as [Inception-v3](http://pytorch.org/docs/master/torchvision/models.html#inception-v3), [ResNet-50](http://pytorch.org/docs/master/torchvision/models.html#id3), etc).  Please use the code cell below to test other pre-trained PyTorch models.  If you decide to pursue this _optional_ task, report performance on `human_files_short` and `dog_files_short`.

In [ ]:
### (Optional) 
### TODO: Report the performance of another pre-trained network.
### Feel free to use as many code cells as needed.

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 10%.  In Step 4 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dog_images/train`, `dog_images/valid`, and `dog_images/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [ ]:
import numpy as np
from glob import glob ## for paths
d = "/kaggle/input/dog-breed-and-human-dataset"

# load filenames for human and dog images
human_files = np.array(glob(d+"/lfw/*/*"))
dog_files = np.array(glob(d+"/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

In [ ]:
!pwd
!ls
!cd "images/"




In [ ]:
!ls

In [ ]:
import os
from torchvision import datasets, transforms

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# train_dir = "/data/dog_images/train"
# valid_dir = "/data/dog_images/valid"
# test_dir  = "/data/dog_images/test"



train_dir = d+"/dogImages/train"
valid_dir = d+"/dogImages/valid"
test_dir  = d+"/dogImages/test"

transform = transforms.Compose([
    transforms.Scale((128,128)),
        transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.ImageFolder(train_dir, transform = transform)
valid_data = datasets.ImageFolder(valid_dir, transform = transform)
test_data = datasets.ImageFolder(test_dir, transform = transform)

print("No of training samples : {}".format(len(train_data)))
print("No of validation samples : {}".format(len(valid_data)))
print("No of testing samples : {}".format(len(test_data)))

In [ ]:
import torch

batch_size=30
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size,
                                           num_workers = num_workers ,
                                          shuffle = True)


valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size,
                                           num_workers = num_workers ,
                                          shuffle = True)


test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size,
                                          num_workers = num_workers ,
                                          shuffle = True)
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
# check if CUDA is available
use_cuda = torch.cuda.is_available()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 20 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
#     ax.set_title(classes[labels[idx]])

**Question 3:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**:

### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  Use the template in the code cell below.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
         # max pooling layer
        self.pool = nn.MaxPool2d(2,2)
    
        # convolutional layer
        self.conv1 = nn.Conv2d(3,  8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        ##two linear layers
        self.fc1 = nn.Linear(16*16*32,1000)
        self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500,133)
        
        ##dropouts
        self.dropout  = nn.Dropout(0.3)
    
    def forward(self, x):
        ## Define forward behavior
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
#         x = self.pool(F.relu(self.conv4(x)))
#         x = self.pool(F.relu(self.conv5(x)))
        
        ## flatten last conv output
        x = x.view(-1,16*16*32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x


#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print(model_scratch)
# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

In [ ]:

# instantiate the CNN
model_scratch = Net()
print(model_scratch)
# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__ 

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(),lr=0.05,momentum = 0.8)
# optimizer_scratch = optim.SGD(model_scratch.parameters(), lr = 2)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            optimizer.zero_grad()
            output = model(data)
#             print("output = ",output)

            loss = criterion(output,target)

            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
#             train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output,target)
            valid_loss += loss.item() * data.size(0)
            
        train_loss  = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)


            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss<valid_loss_min:
            valid_loss_min = valid_loss
            print("validation loss deacreased to {}".format(valid_loss_min))
            torch.save(model.state_dict(),save_path)
    # return trained model
    return model


In [ ]:

loaders_scratch = {"train":train_loader,'valid':valid_loader}
# train the model
save_path = "model_scratch.pt"
model_scratch = train(20, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, save_path)

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images.  Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 10%.

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))


In [ ]:

# call test function    
loaders_scratch = {"test":test_loader}
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively). 

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [ ]:
import numpy as np
from glob import glob ## for paths
d = "/kaggle/input/dog-breed-and-human-dataset"

# load filenames for human and dog images
human_files = np.array(glob(d+"/lfw/*/*"))
dog_files = np.array(glob(d+"/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

In [ ]:
import os
import torch
import torchvision
from torchvision import datasets, transforms

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_dir = d+"/dogImages/train"
valid_dir = d+"/dogImages/valid"
test_dir  = d+"/dogImages/test"

transform = transforms.Compose([
    transforms.Scale((224,224)),
        transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.ImageFolder(train_dir, transform = transform)
valid_data = datasets.ImageFolder(valid_dir, transform = transform)
test_data = datasets.ImageFolder(test_dir, transform = transform)

print("No of training samples : {}".format(len(train_data)))
print("No of validation samples : {}".format(len(valid_data)))
print("No of testing samples : {}".format(len(test_data)))

In [ ]:
import torch

batch_size=30
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size,
                                           num_workers = num_workers ,
                                          shuffle = True)


valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size,
                                           num_workers = num_workers ,
                                          shuffle = True)


test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size,
                                          num_workers = num_workers ,
                                          shuffle = True)
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
# check if CUDA is available
use_cuda = torch.cuda.is_available()


In [ ]:
print(type(train_loader))
print(train_data.classes)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 20 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
#     ax.set_title(classes[labels[idx]])

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [ ]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture 
# model_transfer  = models.resnet50(pretrained =True)
model_transfer  = models.vgg16(pretrained = True)


In [ ]:
print(model_transfer)
import torch


use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()

In [ ]:
# print(model_transfer.fc)
# last_in= model_transfer.fc.in_features
# print(last_in)
# model_transfer.fc = nn.Linear(last_in,133,bias = True)


    
for param in model_transfer.features.parameters():
    param.required_grad = False
    
model_transfer.classifier[6] = nn.Linear(4096,133,bias = True)

if use_cuda:
    model_transfer = model_transfer.cuda()

In [ ]:

# print(model_transfer.fc.in_features)
# print(model_transfer.fc.out_features)
# # print(model_transfer.keys)
# for param in model_transfer.parameters():
#     param.required_grad = False
# for param in model_transfer.fc.parameters():
#     param.requird_grad = True

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [ ]:

import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr = 0.001)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# train the model
n_epochs = 15
loaders_transfer ={'train':train_loader,'valid':valid_loader}
print(use_cuda)
print(torch.cuda.is_available())
model_transfer.cuda()
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
# model_transfer.load_state_dict(torch.load('model_transfer.pt'))

In [ ]:
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))



In [ ]:
loaders_transfer={"test":test_loader}
model_transfer.cuda()
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.
# data_transfer={'train':train_loader}
# list of class names by index, i.e. a name can be accessed like class_names[0]

# class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]
class_names = [item[4:].replace("_", " ") for item in train_data.classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    trans =  transforms.Compose([
    transforms.Scale((224,224)),
    transforms.ToTensor(),

])

    x  = Image.open(img_path)
    x = trans(x)
    x.unsqueeze_(0)
    x.to('cpu')
    model_transfer.to('cpu')
    y = model_transfer(x)
    pred = y.data.max(1, keepdim=True)[1]
    name = class_names[pred]
    return name

---
<a id='step5'></a>
## Step 5: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `human_detector` functions developed above.  You are __required__ to use your CNN from Step 4 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](images/sample_human_output.png)


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    
    ##test human or dog
    dog_status  = dog_detector(img_path)
    #human/dog test result
    if dog_status:
        print("We found a Dog")
    else:
        print("We found a human")
        
    ##find dog_breed human/dog
    dog_breed = predict_breed_transfer(img_path)
    
    ##breed result
    if dog_status:
        
        print("The dog's breed is : {}".format(dog_breed))
    else:
        print("This human matches {} breed of dogs XD !".format(dog_breed))
        
        


---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ (Three possible points for improvement)

In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.

## suggested code, below
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)